In [1]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows

In [2]:


in_path = '/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/classification_outputs/'
input_filename = 'gm_treeclassification_NBART_15_-38_ausalps3.nc'

out_path = '/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/classification_outputs/tiny_tiles/'
output_filename = 'tile_{}-{}.nc'


In [8]:

def get_tiles(ds, width=500, height=500):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform



In [9]:

with rio.open(os.path.join(in_path, input_filename)) as inds:
    tile_width, tile_height = 500, 500

    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        print(window)
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
        with rio.open(outpath, 'w', driver='netCDF', **meta) as outds:
            outds.write(inds.read(window=window))

Window(col_off=0, row_off=0, width=500, height=500)


/g/data/u46/users/ck9738/miniconda3/envs/kerasenv/lib/python3.6/site-packages/rasterio/__init__.py:217: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, **kwargs)


TypeError: open() got multiple values for keyword argument 'driver'